In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import pyrtools as pt
import plenoptic as po
import scipy.io as sio
import seaborn as sns

torch.__version__
numpy = lambda x: x.detach().cpu().numpy().squeeze()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [2]:
# image = plt.imread('../data/flowers.jpg').astype(float).mean(axis=2)
image = plt.imread('../data/metal.pgm').astype(float)
print(image.shape)
im0 = torch.tensor(image,requires_grad=True,dtype = torch.float32).squeeze().unsqueeze(0).unsqueeze(0)

model = po.simulate.Texture_Statistics(image.shape)
y = model.forward(im0)

(256, 256)
torch.Size([9, 9, 5])
tensor([[  44.8164,  292.7126,  588.4286,  904.1422, 1220.5394, 1473.2025],
        [ 338.6368,  611.4792,  898.7686, 1247.7488, 1655.6799, 1951.3042],
        [ 675.3194,  919.8493, 1204.1321, 1731.2123, 2345.8579, 2584.8350],
        [1022.1216, 1269.2629, 1706.4189, 2590.9409, 3363.1631, 3258.0859],
        [1346.4957, 1675.0886, 2303.3701, 3336.8457, 3917.0969, 3336.8462],
        [1581.7891, 1975.8948, 2564.5764, 3258.0859, 3363.1631, 2590.9412]],
       grad_fn=<SelectBackward>)


In [ ]:
s = po.simulate.Steerable_Pyramid_Freq(image.shape, 4,
            3, is_complex=True, store_unoriented_bands=True,
                                          return_list=True)

In [11]:
pyr = s.forward(im0)

In [12]:
pyr[1].shape

torch.Size([1, 1, 256, 256, 2])